In [1]:
def distance(str1, str2):
  d=dict()
  for i in range(len(str1)+1):
     d[i]=dict()
     d[i][0]=i
  for i in range(len(str2)+1):
     d[0][i] = i
  for i in range(1, len(str1)+1):
     for j in range(1, len(str2)+1):
        d[i][j] = min(d[i][j-1]+1, d[i-1][j]+1, d[i-1][j-1]+(not str1[i-1] == str2[j-1]))
  print(d)
  return d[len(str1)][len(str2)]

In [2]:
distance('hola', 'saas')

{0: {0: 0, 1: 1, 2: 2, 3: 3, 4: 4}, 1: {0: 1, 1: 1, 2: 2, 3: 3, 4: 4}, 2: {0: 2, 1: 2, 2: 2, 3: 3, 4: 4}, 3: {0: 3, 1: 3, 2: 3, 3: 3, 4: 4}, 4: {0: 4, 1: 4, 2: 3, 3: 3, 4: 4}}


4

In [2]:
import numpy as np 

from tarea1.edit_distances import matriz
from tarea1.edit_distances import dp_levenshtein_backwards, dp_intermediate_damerau_backwards, dp_restricted_damerau_backwards

In [4]:
for term,ref in [("campos","ejemplo"), ("reconocibles","irreconocible"), ("hola", "holas")]:
    print(term,ref,dp_intermediate_damerau_backwards(term,ref))
    print(matriz(term,ref))

campos ejemplo 5.0
[[1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1]
 [1 1 1 0 1 1 1]
 [1 1 1 1 0 1 1]
 [1 1 1 1 1 1 0]
 [1 1 1 1 1 1 1]]
reconocibles irreconocible 3.0
[[1 0 0 1 1 1 1 1 1 1 1 1 1]
 [1 1 1 0 1 1 1 1 1 1 1 1 0]
 [1 1 1 1 0 1 1 1 0 1 1 1 1]
 [1 1 1 1 1 0 1 0 1 1 1 1 1]
 [1 1 1 1 1 1 0 1 1 1 1 1 1]
 [1 1 1 1 1 0 1 0 1 1 1 1 1]
 [1 1 1 1 0 1 1 1 0 1 1 1 1]
 [0 1 1 1 1 1 1 1 1 0 1 1 1]
 [1 1 1 1 1 1 1 1 1 1 0 1 1]
 [1 1 1 1 1 1 1 1 1 1 1 0 1]
 [1 1 1 0 1 1 1 1 1 1 1 1 0]
 [1 1 1 1 1 1 1 1 1 1 1 1 1]]
hola holas 1.0
[[0 1 1 1 1]
 [1 0 1 1 1]
 [1 1 0 1 1]
 [1 1 1 0 1]]
